## Contenu
#### [1. PART one:](#1)
- [1.1 Preparation de données:](#1.1)
- [1.2 L'entrainement de modéle:](#1.2)
- [1.3 La prédiction de modéle:](#1.3)

#### [2. PART two:](#2)
- [2.1 Le preprocessing:](#2.1)
- [2.1 La gènèration du texte:](#2.2)
- [2.1 L' autocompletion de texte :](#2.3)

#### [3. PART tree:](#3)

<a name='1'></a>
## **I. PART one:**

<a name='1.1'></a>
1. La methode prepare_data:

Mounted at /content/drive


In [ ]:
# Telecharger le module de contractions
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 18.0 MB/s eta 0:00:00


In [ ]:
# Importer les modules
import re
import numpy as np
from collections import defaultdict
import contractions
from nltk.corpus import wordnet
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import words as nltk_words
import math
import pandas as pd
import threading
from pathlib import Path
import argparse
import sys
import os
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
# Fonction de prepare_data
def prepare_data(infile, ngram_size, UNK_threshold=1):
    # Initialiser word counts dictionary
    word_counts = defaultdict(int)
    # Lire le corpus
    with open(infile, 'r', encoding='utf-8') as file:
        corpus = file.read().lower()
    # supprimer les tags htmls
    corpus = re.sub(r'<.*?>', '', corpus)
    # supprimer les URLS
    corpus = re.sub(r'https?:?//\S+', '', corpus)

    # Fixer les contractions
    expanded_corpus = contractions.fix(corpus)

    # Tokenizer les phrases
    sentences = expanded_corpus.split('\n')

    # Le vocabulaire d'anglais
    english_vocab = set(w.lower() for w in nltk_words.words())

    # Tokenizer les mots et compter leurs frequences
    for sentence in sentences:
        # Tokenizer la phrase en mots
        sentence_tokens = nltk.word_tokenize(sentence)

        # les frequences de mots
        for token in sentence_tokens:
            if token in english_vocab:
                word_counts[token] += 1

    # Collecter les mots qui vonts etre remplacées avec <UNK> et ajouter les mots dans le vocabulaire

    words_to_replace = set()
    vocab = set()
    for word, count in word_counts.items():
        if count < UNK_threshold:
            words_to_replace.add(word)
        else:
            vocab.add(word)

    # Replacer les mots with <UNK> and filtrer les mots non anglais
    processed_tokens = []
    for sentence in sentences:
        # Tokenize the sentence into words
        sentence_tokens = nltk.word_tokenize(sentence)

        # Replace rare words with <UNK> and filter non-English words
        sentence_tokens_with_unk = []
        for token in sentence_tokens:
            if token.isalpha():  # Check if the token is alphabetic
               if token in vocab:  # Check if the token is in the vocabulary
                  sentence_tokens_with_unk.append(token)
               else:
                  sentence_tokens_with_unk.append('<UNK>')  # Replace rare words with <UNK>
            else:
                sentence_tokens_with_unk.append(token)  # Preserve non-alphabetic characters
        # Add sentence boundary markers
        sentence_tokens_with_unk.insert(0, '<s> ')  # Add <s> tag at the beginning
        sentence_tokens_with_unk.append(' </s>')    # Add </s> tag at the end

        processed_tokens.append(' '.join(sentence_tokens_with_unk))

    # Add an other  start token in the begining if ngram_size is 3
    if ngram_size == 2:
        pass
    elif ngram_size == 3:
        start_token = '<s> '  # Start token for trigram model
        processed_tokens.insert(0, start_token)
    else:
        raise ValueError("Unsupported ngram_size. Only 2 and 3 are supported.")
    # Return preprocessed corpus
    return ' '.join(processed_tokens), vocab,word_counts

# Example usage
infile = 'test.txt'
preprocessed_corpus, vocab,word_counts = prepare_data(infile,2)
print("Preprocessed corpus:", preprocessed_corpus)
print("vocab:", vocab)


Preprocessed corpus: <s>  how are you ? by the way thanks for the <UNK> . you going to be in <UNK> <UNK> soon ? love to see you . been way , way too long .  </s> <s>  when you meet someone special ... you will know . your heart will beat more rapidly and you will smile for no reason .  </s> <s>  they have decided its more fun if i do not .  </s> <s>  so tired d ; <UNK> <UNK> tag & ran a lot d ; <UNK> going to sleep like in 5 <UNK> ; )  </s> <s>  <UNK> from a complete stranger ! made my birthday even better : )  </s> <s>  first <UNK> game ever ! <UNK> field is gorgeous . this is perfect . go <UNK> go !  </s> <s>  i no ! i get another day off from <UNK> due to the wonderful snow ( : and this wakes me up ... damn thing  </s> <s>  i am coo ... <UNK> at work <UNK> tired r you ever in <UNK>  </s> <s>  the new <UNK> commercial ... hehe love at first sight  </s> <s>  we need to reconnect this week  </s> <s>  i always wonder how the <UNK> on the <UNK> <UNK> learned to talk so fast ! ? all i hea

<a name='1.2'></a>
2. La méthode train:

In [ ]:
def train(infile, ngram_size,k=1):
    # Prétraitement du corpus
    preprocessed_corpus, vocab, word_counts = prepare_data(infile, ngram_size)
    bigram_counts = defaultdict(int)
    trigram_counts = defaultdict(int)

    # Calcul des fréquences des n-grammes
    if ngram_size == 2:
        ngram_counts = bigram_counts
    elif ngram_size == 3:
        ngram_counts = trigram_counts
    else:
        raise ValueError("Unsupported ngram_size. Only 2 and 3 are supported.")
    #print(preprocessed_corpus)
    tokens = preprocessed_corpus.split()

    # Iterating over tokens to count n-grams
    for i in range(len(tokens) - ngram_size + 1):
        ngram = tuple(tokens[i:i + ngram_size])
        ngram_counts[ngram] += 1
    vocabulary=set(tokens) # Contient tous les diffrents tockens mais cette foi ci avec les tags <s> et </s>
    # Lissage add-k
    #print(vocabulary)
    vocab_size = len(vocabulary)-2 # Exclure les tags <s> et </s>
    #print(vocab_size)
    for ngram, count in ngram_counts.items():
        denominator = word_counts[ngram[:-1]] + (k * vocab_size)
        ngram_counts[ngram] = (count + k) / denominator

    # Transformation des probabilités en logarithmes
    for ngram, probability in ngram_counts.items():
        ngram_counts[ngram] = math.log(probability)

    return ngram_counts,vocabulary


In [ ]:
bigram_probabilities=train('ngramv1.test',2)
#print("bigram_probabilities",bigram_probabilities)
trigram_probabilities=train('ngramv1.test',3)
#print("trigram_probabilities",trigram_probabilities)

<a name='1.3'></a>
3. La methode predict_ngram:

In [ ]:
def predict_ngram(sentence, ngram_size):
        file_path = "sentence.txt"
        with open(file_path, 'w', encoding='utf-8') as file:
             file.write(sentence)
        probabilities,vocab=train(file_path,ngram_size)
        probability = sum(probabilities.values())
        #print("La probabilité de la phrase est: ")
        return probability

# La probabilite calculé ici est la somme des logs des probabilites de chaque probabilite p(wi/wi-1) ou p(wi/wi-1wi-2)

Test de fonction

In [ ]:
# Example usage:
sentence = "This is a sample sentence !!!!!"
predict_ngram(sentence,2)

-7.872607577470724

4. La methode test_perplexity:

In [ ]:
def test_perplexity(test_file, ngram_size=2):
        with open(test_file, 'r', encoding='utf-8') as file:
            test_corpus = file.read().lower()
        print(test_corpus)
        # Initialisation de la perplexité
        total_log_prob = 0
        total_words = 0

        # Calcul de la perplexité pour chaque phrase dans le corpus de test
        for sentence in test_corpus.split('\n'):
           if sentence:
                print(sentence)
                # Calcul de la probabilité de la phrase
                log_prob = predict_ngram(sentence, ngram_size)
                total_log_prob += log_prob
                total_words += len(sentence.split())+1 # On ajoute 1 pour chaque phrase en comptant la tage </s>


        # Calcul de la perplexité
        perplexity = math.exp(total_log_prob)**(-1/float(total_words))
        return perplexity

# Exemple d'utilisation :
perplexity_1 = test_perplexity(test_file="ngramv1.test", ngram_size=2)
print("Perplexity:", perplexity_1)

# Exemple d'utilisation :
perplexity_2 = test_perplexity(test_file="ngramv1.test", ngram_size=3)
print("Perplexity:", perplexity_2)

say ! 
i like green eggs and ham !
i do ! 
i like them, sam i am !
and i would eat them in a boat .
and i would eat them with a goat .
and i will eat them, in the rain .
and in the dark .
and on a train .
and in a car .
and in a tree .
they are so good, so good, you see !
so i will eat them in a box .
and i will eat them with a fox .
and i will eat them in a house .
and i will eat them with a mouse .
and i will eat them here and there .
say ! 
i will eat them anywhere !
i do so like green eggs and ham !
thank you !
thank you, sam i am .
say ! 
i like green eggs and ham !
i do ! 
i like them, sam i am !
and i would eat them in a boat .
and i would eat them with a goat .
and i will eat them, in the rain .
and in the dark .
and on a train .
and in a car .
and in a tree .
they are so good, so good, you see !
so i will eat them in a box .
and i will eat them with a fox .
and i will eat them in a house .
and i will eat them with a mouse .
and i will eat them here and there .
say ! 
i will ea

<a name='2'></a>
## **II. PART two:**

<a name='2.1'></a>
2.1. Preprocessing et l'entrainment de modèle sur les données de fichier bigdata.txt:

In [ ]:
bigram_probabilities,vocab=train('big_data.txt',2)
#print("bigram_probabilities",bigram_probabilities)
trigram_probabilities,vocab=train('big_data.txt',3)
#print("trigram_probabilities",trigram_probabilities)

#Ici concernant le  vocabulaire contient les memes mots pour les deux modèles

In [ ]:
def save_probabilities_to_file(probabilities, filename):
    with open(filename, 'w') as file:
        for ngram, prob in probabilities.items():
            file.write(f'{ngram}: {prob}\n')


save_probabilities_to_file(bigram_probabilities, 'bigram_probabilities.txt')
save_probabilities_to_file(trigram_probabilities, 'trigram_probabilities.txt')


In [ ]:
def save_vocab_to_file(vocab, filename):
    with open(filename, 'w') as file:
        for word in vocab:
            file.write(f'{word}\n')

# Assuming bigram_vocab and trigram_vocab are lists containing vocabularies
save_vocab_to_file(vocab, 'big_data_vocab.txt')



<a name='2.2'></a>
2.2 La methode generate_text:

In [ ]:
a=(1,2,3)
print(a[:2])

(1, 2)


In [ ]:
def generate_text(trained_model, vocab, ngram_size, max_length=100):
    current_token_bigram = ("<s> ",)
    current_token_trigram = ("<s> ","<s> ",)  # Ensure that the current token is a tuple
    generated_text = []

    while current_token_bigram != ("</s>",) or current_token_trigram!=  ("</s>",) and len(generated_text) < max_length:
        if ngram_size == 2:  # bigram model
            next_token_probs = {
                token[1]: np.exp(prob) for token, prob in trained_model.items() if token[0] == current_token_bigram[0]
            }
        elif ngram_size == 3:  # trigram model
            next_token_probs = {
                token[2]: np.exp(prob) for token, prob in trained_model.items() if token[:2] == current_token_trigram[:2]
            }
        else:
            raise ValueError("Unsupported ngram_size. Only 2 and 3 are supported.")

        if not next_token_probs:
            print("No next word found, stopping generation.")
            break  # Stop generation if no next word found

        # Exclude "<UNK>" token from next_token_probs
        next_token_probs = {token: prob for token, prob in next_token_probs.items() if token != "<UNK>" }

        # Normalize probabilities
        total_prob = sum(next_token_probs.values())
        next_token_probs = {token: prob / total_prob for token, prob in next_token_probs.items()}

        # Sample the next token proportionally to its probabilities
        next_token = np.random.choice(list(next_token_probs.keys()), p=list(next_token_probs.values()))
        generated_text.append(next_token)

        # Update the current token
        if ngram_size==2:
           current_token_bigram= current_token_bigram[1:] + (next_token,)
        if ngram_size==3:
           current_token_trigram= current_token_trigram[1:] + (next_token,)

    return " ".join(generated_text)


In [ ]:
import numpy as np

def generate_text(trained_model, vocab, ngram_size, max_length=100):
    current_token_bigram = ("<s>",)
    current_token_trigram = ("<s>", "<s>")
    generated_text = []

    while (ngram_size == 2 and current_token_bigram != ("</s>",) or ngram_size == 3 and current_token_trigram != ("</s>",)) and len(generated_text) < max_length:
        if ngram_size == 2:  # bigram model
            next_token_probs = {
                token[1]: np.exp(prob) for token, prob in trained_model.items() if token[0] == current_token_bigram[0]
            }
        elif ngram_size == 3:  # trigram model
            next_token_probs = {
                token[2]: np.exp(prob) for token, prob in trained_model.items() if token[:2] == current_token_trigram[:2]
            }
        else:
            raise ValueError("Unsupported ngram_size. Only 2 and 3 are supported.")

        if not next_token_probs:
            print(f"No next word found for context ")
            break  # Stop generation if no next word found

        # Exclude "<UNK>" token from next_token_probs
        next_token_probs = {token: prob for token, prob in next_token_probs.items() if token != "<UNK>"}

        if not next_token_probs:
           print(f"All next words are <UNK> for context")
           break  # Stop generation if all next words are <UNK>

        # Normalize probabilities
        total_prob = sum(next_token_probs.values())
        next_token_probs = {token: prob / total_prob for token, prob in next_token_probs.items()}

        # Sample the next token proportionally to its probabilities
        next_token = np.random.choice(list(next_token_probs.keys()), p=list(next_token_probs.values()))
        generated_text.append(next_token)

        # Update the current token
        if ngram_size == 2:
            current_token_bigram = (next_token,)
        elif ngram_size == 3:
            current_token_trigram = (current_token_trigram[1], next_token)

    return " ".join(generated_text)

# Example usage
# trained_model, vocab, and ngram_size would be defined elsewhere in your code
# generated_text = generate_text(trained_model, vocab, ngram_size)
# print(generated_text)


In [ ]:
generated_text_bigram = generate_text(bigram_probabilities, vocab, ngram_size=2, max_length=100)
print("Texte généré avec un modèle bigramme :\n", generated_text_bigram)


All next words are <UNK> for context
Texte généré avec un modèle bigramme :
 my america is a loaner


In [ ]:
generated_text_trigram = generate_text(trigram_probabilities, vocab, ngram_size=3, max_length=100)
print("Texte généré avec un modèle trigramme :\n", generated_text_trigram)

All next words are <UNK> for context
Texte généré avec un modèle trigramme :
 how come everyone


<a name='2.3'></a>
2.3. La méthode autoComplete:

In [ ]:
def autoComplete(text, ngram_size, bigram_prob, trigram_prob, vocab):
    # Load n-gram probabilities
    if ngram_size == 2:
        probabilities = bigram_prob
    elif ngram_size == 3:
        probabilities = trigram_prob
    else:
        raise ValueError("Unsupported ngram_size. Only 2 and 3 are supported.")

    # Tokenize input text
    tokens = text.split()

    # Extract last (n-1) tokens as the current token
    current_token = tuple(tokens[-(ngram_size-1):])

    # Generate auto-completion
    generated_text = generate_text(probabilities, vocab, ngram_size)

    # Extract the next word from generated text
    next_word = None
    if generated_text:
        generated_tokens = generated_text.split()
        if len(generated_tokens) >= ngram_size:
            next_word = generated_tokens[ngram_size-1]
        else:
            print("Generated text does not contain enough tokens.")
    else:
        print("No text generated.")

    return next_word

# Example usage
if __name__ == "__main__":
    ngram_size = 3
    text = "i want to know  the truth that"
    next_word = autoComplete(text, ngram_size, bigram_probabilities, trigram_probabilities, vocab)
    print("Next word after '{}': {}".format(text, next_word))


All next words are <UNK> for context
Next word after 'i want to know  the truth that': you


<a name='2.4'></a>
2.4. La méthode de correction: (TP1)







In [ ]:
def build_language_model(corpus_file):
    # Prétraitement du texte du corpus
    with open(corpus_file, 'r', encoding='utf-8') as file:
        text = file.read().lower()

    words = re.findall(r'\w+', text)

    # Comptage du nombre d'occurrences de chaque mot
    word_counts = Counter(words)
    total_words = len(words)

    # Calcul des probabilités de chaque mot
    language_model = {word: count / total_words for word, count in word_counts.items()}

    return language_model

def edits1(word):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [left + right[1:] for left, right in splits if right]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right) > 1]
    replaces = [left + c + right[1:] for left, right in splits if right for c in alphabet]
    inserts = [left + c + right for left, right in splits for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

def known_words(words, vocabulary):
    return set(word for word in words if word in vocabulary)

def candidates(word, vocabulary, language_model):
    # Si le mot original est dans le vocabulaire, retournez-le comme seul candidat avec sa probabilité
    if word in language_model:
        return [(word, language_model[word])]

    # Recherchez les mots connus à une distance d'édition de un
    candidates_1 = known_words(edits1(word), vocabulary)
    candidates_1_with_prob = [(c, language_model.get(c, 0)) for c in candidates_1]
    if candidates_1_with_prob:
        return candidates_1_with_prob

    # Recherchez les mots connus à une distance d'édition de deux
    candidates_2 = known_words(edits2(word), vocabulary)
    candidates_2_with_prob = [(c, language_model.get(c, 0)) for c in candidates_2]
    if candidates_2_with_prob:
        return candidates_2_with_prob

    # Si aucune liste de candidats n'est trouvée, retournez simplement le mot original avec une probabilité nulle
    return [(word, 0)]

# Fonction pour construire un modèle de langue bigramme
def build_bigram_language_model(corpus):
    # Concaténer tous les mots du corpus
    all_words = ' '.join(corpus.keys())
    words = all_words.split()

    # Générer les paires de mots consécutifs pour les bigrammes
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]

    # Compter les occurrences de chaque bigramme
    bigram_counts = Counter(bigrams)
    total_bigrams = sum(bigram_counts.values())

    # Calculer les probabilités de chaque bigramme
    bigram_probabilities = {bigram: count / total_bigrams for bigram, count in bigram_counts.items()}

    return bigram_probabilities


# Fonction pour calculer la probabilité d'un candidat en utilisant le modèle de langue bigramme
def calculate_probability(candidate, bigram_model):
    word1, word2 = candidate
    bigram = (word1, word2)
    return bigram_model.get(bigram, 0)

# Fonction pour ordonner les candidats par probabilité
def order_candidates_by_probability(candidates, bigram_model):
    # Triez les candidats en fonction de leurs probabilités calculées
    ordered_candidates = sorted(candidates, key=lambda x: calculate_probability(x, bigram_model), reverse=True)
    return ordered_candidates

# Fonction principale pour le correcteur orthographique utilisant un modèle de langue bigramme
def spell_corrector_bigram(word, vocabulary, language_model, bigram_model):
    # Génération des candidats
    candidates_list = candidates(word, vocabulary, language_model)

    # Ordonner les candidats par probabilité en utilisant le modèle de langue bigramme
    ordered_candidates = order_candidates_by_probability(candidates_list, bigram_model)

    return ordered_candidates

# Exemple d'utilisation
corpus_file = 'big.txt'
language_model = build_language_model(corpus_file)
vocabulary = set(language_model.keys())
# Utilisation de la fonction build_bigram_language_model avec language_model
bigram_model = build_bigram_language_model(language_model)

word = "iant"
print("Candidates for 'iant':", spell_corrector_bigram(word, vocabulary, language_model, bigram_model))



Candidates for 'iant': [('want', 0.0002895341905816231), ('ant', 2.6891720487457255e-06), ('giant', 2.3306157755796287e-05)]


<a name='3'></a>
## **III. PART three:**

In [ ]:
!pip install conllu

In [ ]:
from conllu import parse_incr
from io import open
from sys import argv
import logging




def train_corpus(lang):
	return 'fr_rhapsodie-ud-train.conllu'


def test_corpus(lang):
	return 'fr_rhapsodie-ud-test.conllu'


def prune_sentence(sent):
	return [token for token in sent if type(token['id']) is int]


def conllu_corpus(path):
	data_file = open(path, 'r', encoding='utf-8')
	sents = list(parse_incr(data_file))
	return [prune_sentence(sent) for sent in sents]


In [ ]:
from nltk import FreqDist, WittenBellProbDist, bigrams

START = '<s>'
END = '</s>'
BINS = 1e5


def get_smoothed(mappings):
	# Initialize the smoothed matrix
	smoothed = {}
	tags = set([t for (t,_) in mappings])
	for tag in tags:
		words = [w for (t,w) in mappings if t == tag]
		# Smooth the mapping using Witten-Bell smoothing
		smoothed[tag] = WittenBellProbDist(FreqDist(words), bins=BINS)
	return smoothed


def get_transmission_prob_matrix(train_sents):
	# Initialize the transition matrix
	transitions = []

	for sent in train_sents:
		# Add the first word of each sentence to the START tag
		transitions.append((START, sent[0]['upos']))

		# Add the transition from the previous word to the current word
		for token in bigrams(sent):
			transitions.append((token[0]['upos'], token[1]['upos']))

		# Add the transition from the last word to the END tag
		transitions.append((sent[len(sent) - 1]['upos'], END))

	# Return the smoothed transition matrix
	return get_smoothed(transitions)

def get_emission_prob_matrix(train_sents):
	# Initialize the emission matrix
    emissions = [(token['upos'], token['form']) for sentence in train_sents for token in sentence]

	# Return the smoothed emission matrix
    return get_smoothed(emissions)

In [ ]:
from math import log


def viterbi_algorithm(tags, emission_probs, transition_probs, test_sentence):
	# Initialize the Viterbi matrix and backpointers
	V = [{}]
	backpointers = {}

	# Initialize the first row of the Viterbi matrix
	for tag in tags:
		V[0][tag] = log(transition_probs[START].prob(tag)) + log(emission_probs[tag].prob(test_sentence[0]['form']))
		backpointers[tag] = [tag]

	# Fill in the rest of the Viterbi matrix
	for word_index in range(1,len(test_sentence)):
		V.append({})
		temp_backpointers = {}

		# Find the maximum probability for each tag
		for curr_tag in tags:
			all_probs = []
			for prev_tag in tags:
				prob = V[word_index - 1][prev_tag] + log(transition_probs[prev_tag].prob(curr_tag)) + log(emission_probs[curr_tag].prob(test_sentence[word_index]['form']))
				all_probs.append((prob, prev_tag))
			max_prob, prev_tag = max(all_probs)

			# Add the maximum probability to the Viterbi matrix and the backpointer to the temp backpointers
			V[word_index][curr_tag] = max_prob
			temp_backpointers[curr_tag] = backpointers[prev_tag] + [curr_tag]

		# Update the backpointers
		backpointers = temp_backpointers

	# Find the maximum probability for the END tag
	max_prob, end_tag = max([(V[-1][tag] + log(transition_probs[tag].prob(END)), tag) for tag in tags])

	# Return the most probable sequence of tags
	return backpointers[end_tag]


def get_accuracy(tags, emission_matrix, transition_matrix, test_sents):
	correct = 0
	total = 0

	for sent in test_sents:
		predicted_tags = viterbi_algorithm(tags, emission_matrix, transition_matrix, sent)
		# Get the accuracy for each word in the test sentence based on the predicted tag
		for token_index, token in enumerate(sent):
			if token['upos'] == predicted_tags[token_index]:
				correct += 1
			total += 1
	return correct / total


In [ ]:

def find_accuracy(lang):
	train_sents = conllu_corpus(train_corpus(lang))
	test_sents = conllu_corpus(test_corpus(lang))
	tags = set([token['upos'] for sent in train_sents for token in sent])

	# Get the transition and emission matrices
	transition_matrix = get_transmission_prob_matrix(train_sents)
	emission_matrix = get_emission_prob_matrix(train_sents)

	# Get the accuracy of the model using the Viertbi algorithm
	accuracy = get_accuracy(tags, emission_matrix, transition_matrix, test_sents)


	# Print the accuracy
	match lang:
		case 'en':
			print('English Accuracy: {:.2%}'.format(accuracy))
		case 'fr':
			print('French Accuracy: {:.2%}'.format(accuracy))
		case 'uk':
			print('Ukrainian Accuracy: {:.2%}'.format(accuracy))
		case _:
			print('Accuracy: {:.2%}'.format(accuracy))

	return accuracy

In [ ]:
find_accuracy('fr')

French Accuracy: 91.78%


0.9178082191780822